In [1]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from IPython.display import display, HTML
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
import time

def extract_brand(sentence):
#   ju name extract kar raha osme se first two words are of brand name tu we are extracting that
    words = sentence.split()
    brand = ' '.join(words[:2])
    return brand



driver = webdriver.Chrome()
driver.get("https://www.daraz.pk/")
search_bar = driver.find_element(By.XPATH, '//*[@id="q"]')
search_bar.send_keys('Mobile Phones' + Keys.RETURN)
driver.implicitly_wait(10)
data_list = []


def scrape_page():
    mobile_phones = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]/div[2]/div')
    for mobile_phone in mobile_phones:
        
        name = mobile_phone.find_element(By.XPATH,'//*[@id="id-title"]').text
        price = mobile_phone.find_element(By.XPATH, '//*[@id="id-price"]/div/div[1]').text
        brand = extract_brand(name)
        rating = mobile_phone.find_element(By.XPATH,'//*[@id="id-rating"]/div[1]').text
        data_list.append([name,price,brand,rating])

# Function to go to the next page
def next_page():
        try:
            next_button_locator = (By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[3]/div[2]/div/ul/li[9]/a')
            next_button = WebDriverWait(driver, 40).until(
                EC.presence_of_element_located(next_button_locator) and
                EC.visibility_of_element_located(next_button_locator)
            )
            # ccroll into view the element is clickable
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            # Move to element and click
            ActionChains(driver).move_to_element(next_button).click().perform()

            # Wait for the next page to load
            time.sleep(40)  

            if not next_button.is_enabled():
                print("Last page reached. Stopping further navigation.")
                return False
            return True

        except TimeoutException:
            print("Timeout waiting for the next button to be present and visible.")
            return False
        except WebDriverException as e:
            print(f"Error while navigating to the next page: {str(e)}") 
            return False
   
while True:
    scrape_page()
    if not next_page():
     break
    
column_names = ['Specs','Price','Brand','Ratings']
df = pd.DataFrame(data_list, columns=column_names)
ef = 'extracted.xlsx'
df.to_excel(ef, index=False)
driver.quit()
display(HTML(df.to_html(escape=False)))
